In [1]:
import torch
if torch.cuda.get_device_capability() < (7, 5):
  raise ValueError(f"You got a GPU with capability {torch.cuda.get_device_capability()}, need at least (7, 5)")
else: print("OK")

# Note: this code requires a Turing GPU or newer. Good: T4, RTX 20xx/30xx, A100/Axx; Bad: K80, P100, V100
# Colab gives you T4. If you get older GPUs, please wait or switch to a new account (don't use both at the same time)
%pip install --quiet bitsandbytes==0.35.4 transformers==4.24.0 datasets==2.7.0 accelerate==0.14.0

OK
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import bitsandbytes as bnb
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "bigscience/bloomz-7b1"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto", load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")

# model = AutoModelForCausalLM.from_pretrained(checkpoint, 
#     load_in_8bit=True, device_map='auto',
#     low_cpu_mem_usage=True, torch_dtype=torch.float16, offload_state_dict=True)

/home/tfsservices/miniconda3/envs/nlp_test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# model.eval()

In [4]:
# for module in model.modules():
#     if isinstance(module, bnb.nn.Linear8bitLt):
#         module.state.memory_efficient_backward = True

for param in model.parameters():
    param.requires_grad = False  # freeze the model - train adapters later
    # if param.ndim == 1:
    #     # cast the small parameters (e.g. layernorm) to fp32 for stability
    #     param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
# # model.model.decoder.project_in = lambda x: x.requires_grad_(True)

# # cast model outputs to float32 to unfuck the top-k sampler
# class CastOutputToFloat(nn.Sequential):
#     def forward(self, x): return super().forward(x).to(torch.float32)
    
# model.lm_head = CastOutputToFloat(model.lm_head)

In [5]:
model.modules

<bound method Module.modules of BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 4096)
    (word_embeddings_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-29): 30 x BloomBlock(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=4096, out_features=16384, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=16384, out_features=4096, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((4096,), eps=1e-0

In [86]:
def evaluate(promt: str, device="cuda", max_length = 1024):
    inputs = tokenizer.encode(promt, return_tensors="pt").to(device)

    with torch.cuda.amp.autocast():
        outputs = model.generate(inputs, max_length, early_stopping=False)

    generated = tokenizer.decode(outputs[0])
    print(generated)

In [7]:
def remove_promt(generated: str, promt: str):
    '''
    Remove original promt and specific signs from the model output.
    '''
    promt_length = len(promt)
    output = generated[promt_length:]
    output = output.replace("</s>", "")
    return output

### Translation tasks

In [8]:
def translate(text: str):

    promt = f"Translate '{text}' to English:"
    inputs = tokenizer.encode(promt, return_tensors="pt").to('cuda')

    max_length = len(promt) * 2
    with torch.cuda.amp.autocast():
        outputs = model.generate(inputs, max_length, early_stopping=True, temperature=1)

    generated = tokenizer.decode(outputs[0])
    generated = remove_promt(promt=promt, generated=generated)
    print(generated)

In [9]:
french_sample = "Maître Corbeau, sur un arbre perché,\nTenait en son bec un fromage."
translate(french_sample) # French

 The Raven, perched on a tree, held in its beak a cheese.


In [10]:
# Talk to the world, do it in Unicode. The 10th Unicode International Conference will be held in Mainz, Germany, from 10 to 12 March 1997. Register now. The conference brings together experts from across the industry to cover areas such as: - Internet and Unicode, internationalization and localization, implementation of Unicode in operating systems and applications, fonts, character arrays, multilingual computing.

korean_sample_long = "세계를 향한 대화, 유니코드로 하십시오. 제10회 유니코드 국제 회의가 1997년 3월 10일부터 12일까지 독일의 마인즈에서 열립니다. 지금 등록하십시오. 이 회의에서는 업계 전반의 전문가들이 함께 모여 다음과 같은 분야를 다룹니다. - 인터넷과 유니코드, 국제화와 지역화, 운영 체제와 응용 프로그램에서 유니코드의 구현, 글꼴, 문자 배열, 다국어 컴퓨팅."

translate(korean_sample_long) 

 The world conference on 'Code, the universal language' will be held from March 10 to 12, 1997 in Seoul, South Korea. Now, register. This conference will focus on the following topics: Internet and Code, Globalization and Regionalization, Economy and Application Programming with Code.


In [11]:
# Korean: I have lived in Korea for seven years.

konean_sample_short = "저는 7년 동안 한국에서 살았어요."
translate(konean_sample_short)

 I have lived in Korea for seven years.


In [12]:
# Japanese: The side business is responsible for the clan's closure, and the company is transferred to the company. Tokujun Shinjutsu Jion The anti-accompaniment tale thought that the swinger would beat the north courtyard. Shinba Shikibu Ashiro Gakuo completed the quasi-secular sculpture method. As a result of the judgment, to recognize the pattern of the rock face on the same rock face. Reclassification of Medicine, Kukan Daikyo and Refinement.

japanese_sample_short = "副業は一族の廃業を担い、会社は会社に譲渡される"
japanese_sample = "副業は一族の廃業を担い、会社は会社に譲渡される. Tokujun Shinjutsu Jion 反伴奏の物語は、ブランコを投げる人が北の中庭を打ち負かすと考えていました. 新馬式部安代学王が準世俗彫刻法を完成. 判定の結果、同じ岩肌にある岩肌の模様を認識すること。 医学の再分類、空間大教と洗練."

translate(japanese_sample)

 The side business is the family business, and the company is given to the company. Tokujun Shinjutsu Jion is the story of a man who throws a broom at people in the north yard. The new master of the Shinjutsu school has completed the Shinjutsu. The result of the test is that you can recognize the model of the same rock. The reclassification of medicine, the space of the great teacher and the simplicity.


In [13]:
# Everyone has a plan til they get punched in the face.
chinese_sample_short = "每个人都有他的作战策略，直到脸上中了一拳"

translate(chinese_sample_short)

 Everyone has his own strategy, until he gets hit in the face.


In [14]:
## Chinese: Junior student Leli. The small plan is high; he was born in a house that is too unsocial, where the sky is heavy with two powers, and the heart is alone.
chinese_sample = "小科生樂利。小小計高；他生太不社的府一廣天重以二權來何、心一獨生自"
translate(chinese_sample) 

 "Lucky little kid. Little boy, you are so smart; you are so lucky to have such a big family, two parents, and a heart of your own."


### Language detection task

In [15]:
def detect_language(text: str):

    promt = f"Language of 'I love you' is English. Language of '{text}':"
    inputs = tokenizer.encode(promt, return_tensors="pt").to('cuda')

    max_new_tokens = 32
    with torch.cuda.amp.autocast():
        outputs = model.generate(inputs, max_new_tokens=max_new_tokens, early_stopping=True, temperature=0)

    generated = tokenizer.decode(outputs[0])
    generated = remove_promt(promt=promt, generated=generated)
    print(generated)

In [16]:
detect_language(french_sample) # French

 French


In [17]:
detect_language(konean_sample_short)

 Korean


In [18]:
detect_language(korean_sample_long)

 Korean


In [19]:
detect_language(japanese_sample_short)

 Japanese


In [20]:
detect_language(japanese_sample) 

 English


In [21]:
detect_language(chinese_sample_short)

 Chinese


In [22]:
detect_language(chinese_sample)

 Japanese


### Spell and grammar check

In [67]:
def correct(text: str):

    promt = f"Poor English: '{text }' . Correct English:"
    inputs = tokenizer.encode(promt, return_tensors="pt").to('cuda')

    max_length = len(text) * 2
    with torch.cuda.amp.autocast():
        outputs = model.generate(inputs, max_length=max_length, early_stopping=True)

    generated = tokenizer.decode(outputs[0])
    generated = remove_promt(promt=promt, generated=generated)
    print(generated)

In [ ]:
# text = "Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum."

# # text = "Lorem Ipsum is simply dummy text of the printing and typesetting industry."

# evaluate(f"Article:'{text}'. \n" +
#          "Q: what is the Lorem? \n" +
#          "A:")

In [68]:
correct("She no went to the market.")

She didn't go to the market.


In [70]:
correct("She didnt went to the market for AMAT/APPLIED MATERIALS P5000. ")

She didn't go to the market for AMAT/APPLIED MATERIALS P5000.


In [71]:
correct("She didnt went to the market for AMAT/APPLIED MATERIALS P5000.")

She didn't go to the market for AMAT/APPLIED MATERIALS P5000. Yes


In [72]:
correct("She didnt went to the market for AMAT P5000.")

She didn't go to the market for AMAT P5000. Yes


In [73]:
correct("She didnt went to the market for AMAT P5000. ")

She didn't go to the market for the AMAT P5000.


In [76]:
correct("Im a yung boy.")

'I'm a young boy.'


In [78]:
correct("futboll is an a greates gane in the world!")

'football is the greatest game in the world!'


### Summarization

### Other

In [87]:
evaluate("Human: Hi there! AI:")

Human: Hi there! AI: Hello!</s>


In [88]:
evaluate("Human: Who are you? \nAI:")

Human: Who are you? 
AI: I'm a robot.</s>


In [89]:
evaluate("Human: Could you help me with listing equipment? \nAI:")

Human: Could you help me with listing equipment? 
AI: Yes, I can.</s>


In [90]:
evaluate("Human: I want to buy AMAT/APPLIED MATERIALS P5000 asap. Do you have it in stock? \nAI:")

Human: I want to buy AMAT/APPLIED MATERIALS P5000 asap. Do you have it in stock? 
AI: Yes, I have it in stock.</s>


In [92]:
evaluate("Human: I want to buy AMAT/APPLIED MATERIALS P5000 asap. What equipment do you have in stock? \nAI:")

Human: I want to buy AMAT/APPLIED MATERIALS P5000 asap. What equipment do you have in stock? 
AI: I have a lot of stock.</s>
